# 1 파이토치 살펴보기

## 1.1 파이토치 모듈

- **torch.nn**

neural network architecture을 구축할 때, network가 갖는 기본 특징은 다음과 같다.

- layer의 수

- 각 layer의 neuron 수

- 그중 학습 가능한 neuron 수 등

다음 예시는 torch.nn 모듈을 사용하지 않고 perceptron을 초기화하는 코드이다.

In [ ]:
import torch
import math

# 단일 perceptron 
# input은 256차원, output은 4차원
# 랜덤값으로 (256x4) 행렬을 initialize한다.
weights = torch.randn(256, 4) / math.sqrt(256)

# weight를 trainable하게 만든다.(autograd)
# 즉, 256x4 matrix 값이 backpropagation을 통해 조정될 수 있게 만든다.
# 추후 tensor의 backward()를 이용해 Jacobian matrix과 chain rule을 이용하여 backpropagation을 시도한다.
weights.requires_grad_()

# 4차원 output을 위한 bias. 이 bias도 trainable하게 설정한다.
bias = torch.zeros(4, requires_grad=True)

주로 사용하는 tensor에 랜덤값을 채우는 function은 다음과 같다.

- `torch.rand()`: 0과 1사이 숫자를 랜덤하게 생성

- `torch.randn()`: mean = 0, standard deviation = 1인 **normal distribution**(정규분포, Gaussian distribution)을 이용해 랜덤값 생성

> `torch.manual_seed()`을 이용해서 사용하는 seed 값을 설정할 수 있다. 

torch.nn 모듈을 사용해 initialize하면 다음과 같다. linear regression model을 쉽게 구현할 수 있다.

```Python
# nn.Linear(input_dim,output_dim)
nn.Linear(256, 4)
```

torch.nn 모듈에는 `torch.nn.functional`이라는 하위 모듈이 있다. 이 하위 모듈은 torch.nn 내부의 모든 function을 포함한다.(이외 다른 하위 모듈은 모두 class이다.)

> loss function, activation function, functional한 방식으로 생성하기 위해 사용될 수 있는 pooling 등

다음은 torch.nn.functional 모듈을 사용한 loss function의 예시다.

In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy
loss = loss_func(model(X), y)

- X: input

- y: target output

- **torch.optim**

`torch.optim` 모듈은 **optimization**(최적화) 과정에 필요한 여러 도구와 기능을 가지고 있다. 

다음은 optim 모듈을 이용해 optimizer를 정의하는 예시다.

In [ ]:
opt = optim.SGD(model.parameters(), lr=lr)

> SGD(Stochastic Gradient Descent, 확률적 경사하강법)

> lr(learning rate, 학습률)

In [ ]:
with torch.no_grad():
    # stochastic gradient descent를 사용해 parameter updata를 적용한다.
    for param in model.parameters(): param -= param.grad * lr
    model.zero_grad()

> 'model.zero_grad()'를 적용하는 이유는 한 번의 iteration이 끝나면 gradient를 0으로 초기화하기 위함이다.(제대로 weight를 update하기 위해서 필요하다.)

optim을 사용하면 다음과 같이 간단하게 작성할 수 있다.

In [ ]:
# parameters를 update
opt.step()

# 
opt.zero_grad()

- **torch.utils.data**

`utils.data` 모듈은 자체 data set을 제공한다. 
또한 내부의 `DatasetLoader` class를 이용하면 data 배치를 편리하게 수행할 수 있다.

우선 data 배치를 수작업으로 했을 때를 보자.

In [ ]:
#epoch가 있다면 loop를 하나 더 감싸게 된다.
for i in range((n-1)//bs + 1):
    start_i = i * bs
    end_i = start_i + bs
    
    x_batch = x_train[start_i:end_i]
    y_batch = y_train[start_i:end_i]
    pred = model(x_batch)
    loss = loss_func(pred, y_batch)
    #...

> bs(batch size)

대신 utils.data 모듈을 쓰면 다음과 같이 간단하게 code를 작성할 수 있다.

In [ ]:
from torch.utils.data import (TensorDataset, DataLoader)
train_dataset = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=bs)

for x_batch, y_batch in train_dataloader:
    pred = model(x_batch)

- **torch.tensor**

`tensor` 모듈은 numpy와 비슷하게 생각하면 된다. math function을 연산할 수 있고 GPU를 통해 speedup이 가능한 n차원 array이다.

> computational graph(계산 그래프)나 gradient를 기록하는 데 사용할 수도 있다.

PyTorch에서 tensor는 연속된 memory에 저장된 숫자 data의 1차원 array의 뷰로 구현된다. 이를 storage instance라고 한다.

다음은 tensor를 인스턴스화하는 예시다.

In [ ]:
points = torch.tensor([1.0, 4.0, 2.0, 1.0, 3.0, 5.0])

첫 번째 항목을 가져오려면 다음과 같이 하면 된다.

In [ ]:
# 첫 번째 항목(1차원 array)을 조회한다.
float(points[0])

# tensor의 모양을 확인한다.
points.shape

storage instance를 출력하는 `storage` property를 이용해서 조회할 수도 있다. 다음 예시를 보자.

In [ ]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])

points.storage()

tensor를 구현하기 위해 사용된 정보(properties)들을 다음과 같이 조회할 수 있다.

In [ ]:
# tensor size 확인
# 참고로 size를 모두 곱하면 storage instance의 총 길이를 알 수 있다.
torch.Size([3, 2])

# offset 확인
# offset이란 storage array에서 tensor의 첫 번째 element의 index를 의미한다.
points.storage_offset()

# 다음과 같이 이용할 수 있다.
# points[1]은 [2.0]이고, 이는 storage array에서 index 2에 위치한다.
points[1].storage_offset()

# stride 확인
# 각 차원에서 tensor의 다음 element로 접근하기 위해 건너뛰어야 할 element 개수를 나타낸다.
points.stride()

다음과 같이 tensor에서 사용할 data type을 지정할 수 있다.

In [ ]:
points = torch.tensor([[1.0, 2.0], [3.0, 4.0]], dtype=torch.float32)

또한 PyTorch의 tensor는 data를 저장할 장치를 정해야 한다.

- `device='cpu'`: CPU에 할당(default)

- `device='cuda'`: GPU에 할당

> 현재 PyTorch는 CUDA를 지원하는 GPU만 지원한다.

In [ ]:
h_points = torch.tensor([[1.0, 2.0], [3.0, 4.0]], dtype=torch.float32, device='cpu')

다른 장치로 tensor를 copy하는 방법은 다음과 같다.

In [ ]:
d_points = h_points.to(device='cuda')

# GPU가 여러 대 존재한다면 다음과 같이 지정할 수도 있다.
d_points_2 = h_points.to(device='cuda:0')

---

## 1.2 backpropagation시키기

1. autograd 활성화시키기

- 방법 1: tensor 생성 때 parameter로 `requires_grad=True` 넘겨 주기



In [1]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

NameError: name 'torch' is not defined